In [ ]:
###############################################################################
import aliases

In [ ]:
from everest.ptolemaic.chora import *
from everest.ptolemaic.ptolemaic import Ptolemaic

In [ ]:
from everest.utilities import caching as _caching


@Chora.decorate
class MyClass(Ptolemaic):

    __slots__ = ('dct',)

    def __init__(self, dct, /):
        self.dct = dct

    @property
    @_caching.soft_cache()
    def chora(self, /):
        return self.Chora()

    def retrieve(self, index):
        return self.dct[index]

    def __contains__(self, arg, /):
        return arg in self.dct

    def _repr(self, /):
        return self.dct

    def get_epitaph(self, /):
        return self.taphonomy.custom_epitaph(
            '$A(a)',
            dict(A=type(self)._ptolemaic_class__, a=self.dct),
            )

In [ ]:
out = MyClass({'a': 1, 'b': 2})[Chora()]
out

In [ ]:
out['b']

In [ ]:
out

In [ ]:
out is out

In [ ]:
out[:]

In [ ]:
out.chora.getitem(out, slice(None))

In [ ]:
out[:] is out

In [ ]:
out2 = out[Chora()]

In [ ]:
out2['a']